# 06. PyTorch Transfer Learning
Taking the parameters of what one model has learned on another dataset and applying it on our own problem...

Pretrained models are also called as foundation models.

In [ ]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)  # want 0.13+

2.2.1+cu121
0.17.1+cu121


In [ ]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Total 4056 (delta 0), reused 0 (delta 0), pack-reused 4056
Receiving objects: 100% (4056/4056), 646.90 MiB | 23.50 MiB/s, done.
Resolving deltas: 100% (2371/2371), done.
Updating files: 100% (248/248), done.


In [ ]:
# Setup device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## 1. Get Data
We need our pizza, sushi and steak data to build a transfer model on.

In [ ]:
import os
import zipfile

from pathlib import Path

import requests

# Setup data path and image path
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'  # Images from a subset of classes from the Food101 dataset.

# If the image folder doesn't exist, download it and create folder...
if image_path.is_dir():
  print(f'{image_path} directory exists, skipping re-download...')
else:
  print(f'Did not find {image_path}, downloading it...')
  image_path.mkdir(parents=True, exist_ok=True)

  # Downloading the data
  with open(data_path/'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    print('Downloading pizza, steak, sushi data...')
    f.write(request.content)

  # Unzip the data
  with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
    print('Unzipping pizza, steak and sushi data')
    zip_ref.extract_all(image_path)

  # Deleting the zip file
  os.remove(data_path/'pizza_steak_sushi.zip')

data/pizza_steak_sushi directory exists, skipping re-download...


In [ ]:
train_dir = image_path / 'train'
test_dir = image_path / 'test'
train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

## 2. Create Datasets and DataLoaders

We'll use the `data_setup.py` module we created earlier in the course. The `create_dataloaders()` function will help us create these.

For transforming the data, `torchvision` 0.13+ has two ways:
1. Manually created transforms - You define what transforms you want your data to go through
2. Automatically created transforms - the transforms for your data are automatically defined by the model you use.

NOTE: When using a pretrained model, it's important that the data (including your own custom data) is transformed in the same way as the data the model was trained on.

### 2.1 Creating a transform for `torchvision.models` (manual creation)

`torchvision.models` contains pretrained models (models ready for transfer learning) right within torchvision.

> All pre-trained models expect input images normalized in the same way i.e. mini-batches of 3-channel RGB images of shape (3xWxH), where W and H are expected to be atleast 224. The images must be loaded into a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. You can use the following transform to normalize.

In [ ]:
from torchvision import transforms

# The mean and std values are the values for the imagenet dataset's distribution.
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Reshape images to 224, 224
    transforms.ToTensor(),  # Get images into range [0, 1]
    normalize
])

In [ ]:
from going_modular.going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,
                                                                               test_dir,
                                                                               manual_transforms,
                                                                               batch_size=32,
                                                                               num_workers=os.cpu_count())
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7bc472feea10>,
 ['pizza', 'steak', 'sushi'])

### 2.2 Creating a transform for `torchvision.models` (auto-creation)

As of `torchvision` 0.13+ there is now support for automatic data transform creation based on the pretrained model weights that you're using.

In [ ]:
# Get a set of pretrained model weights
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT  # "DEFAULT" = best available weights
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [ ]:
# Get the transforms used to create our pretrained weights
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [ ]:
# Create dataloaders from automatic transforms
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,
                                                                               test_dir,
                                                                               auto_transforms,
                                                                               batch_size=32,
                                                                               num_workers=os.cpu_count())
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7bc472f75450>,
 ['pizza', 'steak', 'sushi'])